# IMOVE Time-Series and Big-Data Workshop
In this workshop we will use the time-series functionality of [Pandas](https://pandas.pydata.org/docs/) and [Xarray](http://xarray.pydata.org/en/stable/) to explore some data collected by the [Ocean Observatories Intiative](https://oceanobservatories.org/). Hopefully we will also get a chance to explore [Dask](https://docs.dask.org/en/latest/) and [Dask Delayed](https://docs.dask.org/en/latest/delayed.html) functions to parallelize a data analysis workflow in the cloud. We will be working on the [OOICloud](https://www.ooicloud.org/) [Pangeo](http://pangeo.io/) instance. Further information on using Python to analyze Earth science datasets can be found in the book [Earth and Environmental Data Science](https://earth-env-data-science.github.io/intro) which I have been using to teach Research Computing in the Earth Sciences this semester.

## Bottom pressure data at Axial Seamount
Here we find data using the new OOI Data Explorer and use Pandas [read_csv()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) and [time-series functionality](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html) to plot a smoothed representation of the bottom pressure at Axial Seamount. 

### First import some required packages

### Next find some data on the OOI Data Explorer

Link to new OOI Data Explorer: https://dataexplorer.oceanobservatories.org/

In [ ]:
url = 'http://erddap.dataexplorer.oceanobservatories.org/erddap/tabledap/ooi-rs03ccal-mj03f-05-botpta301.csv?time%2Cbotpres%2Cbotpres_qc_agg%2Cz&time%3E%3D2014-08-29T20%3A59%3A00Z&time%3C%3D2020-11-21T06%3A00%3A00Z'

## Earthquake catalog from the OOI seismic array at Axial Seamount
Here we parse and plot Axial Seamount earthquake catalog data from [William Wilcock's near-real-time automated earthquake location system](http://axial.ocean.washington.edu/). The data we will use is a text file in they HYPO71 output format located here: http://axial.ocean.washington.edu/hypo71.dat.

In [ ]:
eqs_url = 'http://axial.ocean.washington.edu/hypo71.dat'

In [ ]:
col_names = ['ymd', 'hm', 's', 'lat_deg', 'lat_min', 'lon_deg', 'lon_min',
        'depth', 'MW', 'NWR', 'GAP', 'DMIN',  'RMS',  'ERH', 'ERZ', 'ID', 'PMom', 'SMom']

In [ ]:
from datetime import datetime
def parse_hypo_date(ymd, hm, s):
    hour = int(hm.zfill(4)[0:2])
    minute = int(hm.zfill(4)[2:])
    second = float(s)
    if second == 60:
        second = 0
        minute += 1
    if minute == 60:
        minute=0
        hour +=1
    eq_date_str = ('%s%02.0f%02.0f%05.2f' % (ymd, hour, minute, second))
    return datetime.strptime(eq_date_str, '%Y%m%d%H%M%S.%f')

### Mapping eq data
Let's make some maps just because we can.

## OOI Seafloor Camera Data
Now let's look at some video data from the [OOI Seafloor Camera](https://oceanobservatories.org/instrument-class/camhd/) system deployed at Axial Volcano on the Juan de Fuca Ridge. We will make use of the [Pycamhd](https://github.com/tjcrone/pycamhd) library, which can be used to extract frames from the ProRes encoded Quicktime files. These data are hosted on Microsoft's [Azure Open Datasets](https://azure.microsoft.com/en-us/services/open-datasets/).

In [ ]:
dbcamhd_url = 'https://ooiopendata.blob.core.windows.net/camhd/dbcamhd.json'

In [ ]:
def show_image(frame_number):
    plt.rc('figure', figsize=(12, 6))
    plt.rcParams.update({'font.size': 8})
    frame = camhd.get_frame(mov.url, frame_number)
    fig, ax = plt.subplots();
    im1 = ax.imshow(frame);
    plt.yticks(np.arange(0,1081,270))
    plt.xticks(np.arange(0,1921,480))
    plt.title('Deployment: %s    File: %s    Frame: %s' % (mov.deployment, mov['name'], frame_number));